## 빅데이터 실습

### 다나와 무선청소기 상품분석

#### 데이터 수집

##### 셀레니움 사용

In [1]:
import mplcursors
## 필수 라이브러리 사용등록
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib import parse # url encode
from tqdm import tqdm
import pandas as pd
import time

In [2]:
## 웹드라이버로 크롬 오픈
driver = webdriver.Chrome()
url = r'https://search.danawa.com/dsearch.php?k1=%EC%97%90%EC%96%B4%EB%A9%94%EC%9D%B4%EB%93%9C+%EC%8A%A4%EB%A7%88%ED%8A%B8%ED%83%80%EC%9B%8C&module=goods&act=dispMain'
driver.get(url)
time.sleep(5.0)

In [3]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [4]:
prodItems = soup.select('ul.product_list > li.prod_item')

In [5]:
len(prodItems)

6

In [6]:
prodItems[0].select('p.prod_name > a')[0].text.strip()

'에어메이드 스마트타워 AMC-3501A'

In [7]:
prodItems[0].select('div.spec_list')[0].text.strip().replace('\t', '')

'핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 370W / 2024년형 / [구성] 먼지비움 / 충전 / UVC LED / 브러쉬: 바닥 / 침구 / 솔형 / 틈새 / 먼지봉투: 2.5L / [배터리] 사용시간: 40분(최대) / 충전시간: 5시간 / 분리형(1개) / 2500mAh / [청소] LED라이트 / BLDC모터 / [부가] 디스플레이표시 / 헤파필터 / 색상: 화이트 / 무게: 2.5kg / 크기(가로x세로x깊이): 252x1253x185mm'

In [8]:
# 만약에 수가 아닌 문자열 들어왔을때는 False
prodItems[0].select('input')[1].get('value').isdecimal()

True

##### 다나와 무선청소기 웹크롤링 다시
- 가격외에는 안들어오도록 변환

In [9]:
## 검색어, 페이지를 변경하면서 URL 생성함수
def getSearchPageUrl(keyword, page):
    ecKeyword = parse.quote(keyword)
    url = f'https://search.danawa.com/dsearch.php?query={ecKeyword}&originalQuery={ecKeyword}&previousKeyword={ecKeyword}&checkedInfo=N&volumeType=allvs&' + \
          f'page={page}&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&' + \
           'recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&' + \
           'quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'
    return url

In [10]:
## 상품정보 추출하는 함수
def getProdItems(prodItems):
    prodData = []

    for prodItem in prodItems:
        try:
            prodName = prodItem.select('p.prod_name > a')[0].text.strip()  # 상품명 가져오기
            specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t', '') # 상품 스펙목록 가져오기

            if prodItem.select('input')[1].get('value').isdecimal() == True: 
                price = prodItem.select('input')[1].get('value') # 최저가 가져오기
            else:
                price = 0 # 문자열 들어온 것 막음

            prodData.append([prodName, specList, price])
        except:
            pass
    
    return prodData

In [11]:
## 여러페이지 검색후 크롤링하는 작업
driver = webdriver.Chrome()
# 암묵적으로 웹 자원 로드를 위해 3초정도 대기
driver.implicitly_wait(3.0)

keyword = '무선청소기'
startPage = 1
totalPage = 20
prodDataTotal = [] # 최종적으로 저장할 리스트

for page in tqdm(range(startPage, totalPage +1)):
    # 검색 페이지 이동
    url = getSearchPageUrl(keyword, page)
    driver.get(url)
    # 페이지 로딩이 완료될때까지 5초간 대기
    time.sleep(5)

    # 현재 페이지 HTML 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 상품정보 추출
    prodItems = soup.select('ul.product_list > li.prod_item')
    prodItemList = getProdItems(prodItems) # 리스트로 추출하는 함수

    # 추출된 정보를 prodDataTotal 추가
    prodDataTotal += prodItemList   

100%|██████████| 20/20 [03:29<00:00, 10.49s/it]


In [12]:
dfProdDataTotal = pd.DataFrame(prodDataTotal)

In [13]:
dfProdDataTotal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2215 entries, 0 to 2214
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       2215 non-null   object
 1   1       2215 non-null   object
 2   2       2215 non-null   object
dtypes: object(3)
memory usage: 52.0+ KB


In [14]:
dfProdDataTotal.columns = ['제품명','제품스펙','최저가']

In [15]:
dfProdDataTotal.to_excel('./data/다나와_무선청소기_결과.xlsx', index=False)

##### 크롤링 데이터 전처리

In [16]:
# 저장한 엑셀을 재로드
dfProdDanawa = pd.read_excel('./data/다나와_무선청소기_결과.xlsx')
dfProdDanawa.tail()

,제품명,제품스펙,최저가
2210,메이드조이 차량용 미니 청소기 MV-V100,"차량용청소기 / 무선 / [흡입력] 4,000Pa / [최대출력] 120W / BL...",29800
2211,Garrl-231,침구청소기 / 핸디형 / 무선 / 사용시간: 45분(최대) / [기능] 청소모드: ...,19400
2212,GJIE V09,"침구청소기 / 핸디형 / [기능] 청소모드: 온풍, 진동, 흡입 / 진동수: 800...",54300
2213,아이룸 UC-81 PLUS,물걸레청소기 / 회전식 (분당 250회) / 물걸레전용 / 무선 / 소비전력: 85...,75000
2214,SHILONG CMY-01,침구청소기 / 핸디형 / 무선 / 충전시간: 2시간 / [기능] 청소모드: UV살균...,33800


In [17]:
dfProdDanawa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2215 entries, 0 to 2214
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     2215 non-null   object
 1   제품스펙    2205 non-null   object
 2   최저가     2215 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 52.0+ KB


In [18]:
dfProdDanawa.isnull().sum()

제품명      0
제품스펙    10
최저가      0
dtype: int64

In [19]:
# 결측치 검색
condition = dfProdDanawa['제품스펙'].isnull() == True

In [20]:
dfProdDanawa[condition]

,제품명,제품스펙,최저가
1832,스틱 핸디 교체 사용 무선 충전 가성비 진공 청소기 오래가는 좋은 집들이,NaN,0
1833,클래파 스틱형+핸디형 5단계 필터먼지통 고성능 BLDC모터 싸이클론 무선 진공청소기,NaN,0
1915,스틱 핸디 교체 사용 무선 충전 가성비 진공 청소기 오래가는 좋은 집들이,NaN,0
1916,클래파 스틱형+핸디형 5단계 필터먼지통 고성능 BLDC모터 싸이클론 무선 진공청소기,NaN,0
1998,스틱 핸디 교체 사용 무선 충전 가성비 진공 청소기 오래가는 좋은 집들이,NaN,0
1999,클래파 스틱형+핸디형 5단계 필터먼지통 고성능 BLDC모터 싸이클론 무선 진공청소기,NaN,0
2081,스틱 핸디 교체 사용 무선 충전 가성비 진공 청소기 오래가는 좋은 집들이,NaN,0
2082,클래파 스틱형+핸디형 5단계 필터먼지통 고성능 BLDC모터 싸이클론 무선 진공청소기,NaN,0
2164,스틱 핸디 교체 사용 무선 충전 가성비 진공 청소기 오래가는 좋은 집들이,NaN,0
2165,클래파 스틱형+핸디형 5단계 필터먼지통 고성능 BLDC모터 싸이클론 무선 진공청소기,NaN,0


In [21]:
## 결측치가 있는 행(row)을 삭제
dfProdDanawa = dfProdDanawa.dropna(axis=0)

In [22]:
## 최저가가 0인 제품
condition = dfProdDanawa['최저가'] == 0

In [23]:
# 최저가가 0은 제품 제외한 나머지
dfProdDanawa = dfProdDanawa[condition == False]

In [24]:
# 행들이 삭제되면서 인덱스가 꼬임. 인덱스 초기화
dfProdDanawa.reset_index(drop=True, inplace=True)

In [25]:
# 결측치, 이상치를 제거한 최종 DF
# 회사명, 모델명, 카테고리, 사용시간, 흡입력 추출한 결과도 2057개가 필수
dfProdDanawa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     2155 non-null   object
 1   제품스펙    2155 non-null   object
 2   최저가     2155 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 50.6+ KB


In [26]:
# 회사명, 제품명 분리 split(' ', n) n => 자를 공백의 번호
dfProdDanawa['제품명'][:5][0].split(' ', 1)

['LG전자', '오브제컬렉션 코드제로 A9S AX9884']

In [27]:
## 회사명, 모델명 분리 시작
compList = []
modelList = []
count = 0

for temp in dfProdDanawa['제품명']:
    titles = temp.split(' ', 1) # 길이 2 배열 생성
    if (len(titles) > 1):
        compList.append(titles[0]) # 회사명
        modelList.append(titles[1]) # 모델명
    else:
        compList.append('') # 회사명은 비운다
        modelList.append(titles[0]) # 모델명

    count +=1    

In [28]:
len(compList)

2155

In [29]:
len(modelList)

2155

In [30]:
# 스펙목록 데이터
specList = dfProdDanawa.loc[2000, '제품스펙'].split(' / ')

In [31]:
specList

['물걸레청소기',
 '회전식',
 '물걸레전용',
 '무선',
 '소비전력: 30W',
 '[배터리] 리튬이온',
 '[기능] 물분사',
 '각도조절',
 '셀프스탠딩',
 '[부가] 색상: 화이트, 블랙',
 '패드: 극세사(벨크로형)',
 '크기(가로x세로x깊이): 345x1230x165mm']

In [32]:
useTime = ''
suctionPow = ''
for spec in specList:
    if '사용시간' in spec:
        useTime = spec
    elif '흡입력' in spec:
        suctionPow = spec

print(useTime.split(' ')[1].strip())
print(suctionPow.split(' ')[1].strip()) # 흡입력이 없어서 ''경우는 split(' ')에서 예외발생

IndexError: list index out of range

In [ ]:
dfProdDanawa.loc[0, '제품스펙']

In [ ]:
## 위의 테스트를 기반으로 카테고리, 사용시간, 흡입력 추출
categoryList = []
useTimeList = []
suctionPowList = []
count = 0

for spec in dfProdDanawa['제품스펙']:
    # ' / '로 문자열 분리
    specList = spec.split(' / ')
    # 카테고리 추출
    category = specList[0]
    categoryList.append(category)
    # 사용시간, 흡입력 추출
    useTimeVal = None
    suctionPowVal = None

    for temp in specList:
        if '사용시간' in temp:
            useTimeVal = temp.replace('[배터리] ', '') # 1. [배터리]가 값 오류 발생시킴
        elif '흡입력' in temp:
            suctionPowVal = temp

    # 끝난 다음에도 사용시간, 흡입력이 None이면 스펙에 찾을값이 없었음
    if useTimeVal != None:
        useTime = useTimeVal.split(' ')[1].strip().replace('(최대)', '') # 2. (최대) 삭제
    else:
        useTime = ''

    if suctionPowVal != None:
        suctionPow = suctionPowVal.split(' ')[1].strip()
    else:
        suctionPow = ''

    useTimeList.append(useTime)
    suctionPowList.append(suctionPow)
    count += 1
    # print(count)

In [ ]:
len(categoryList)

In [ ]:
len(suctionPowList)

In [ ]:
len(useTimeList)

In [ ]:
time = '1시간20분'
time.split('시간')[-1].split('분')[0]

In [ ]:
## 사용시간 단위를 통일, 
# time = '1시간' ,'30분', '1시간20분'
def convertHourToMin(time):
    try:
        if '시간' in time:
            hour = time.split('시간')[0]
            if '분' in time:
                minute = time.split('시간')[-1].split('분')[0]
            else:
                minute = 0
        else:
            hour = 0
            minute = time.split('분')[0]
        return int(hour)*60 + int(minute)
    except:
        return None

In [ ]:
newUseTimeList = []
for time in useTimeList:
    value = convertHourToMin(time)
    newUseTimeList.append(value)

In [ ]:
len(newUseTimeList)

In [ ]:
## 흡입력 단위 통일 1W = 1AW = 100pa
def convertPow(value):
    try:
        value = value.upper()
        if 'AW' in value or 'W' in value:
            result = value.replace('A', '').replace('W', '').replace(',', '') # A도 삭제, W도 삭제, 1000단위 쉼표 삭제
            result = int(result)
        elif 'PA' in value:
            result = value.replace('PA', '').replace(',', '')  #PA 삭제, 1000단위 쉼표 삭제
            result = int(result) // 100
        else:
            result = None
        
        return result
    except:
        return None

In [ ]:
newSuctionList = []
for power in suctionPowList:
    value = convertPow(power)
    newSuctionList.append(value)

In [ ]:
len(newSuctionList)

In [ ]:
# 최종 데이터 엑셀 저장
dfLast = pd.DataFrame()
dfLast['카테고리'] = categoryList
dfLast['회사명'] = compList
dfLast['제품명'] = modelList
dfLast['가격'] = dfProdDanawa['최저가']
dfLast['사용시간'] = newUseTimeList
dfLast['흡입력'] = newSuctionList

In [ ]:
dfLast.tail()

In [ ]:
dfLast.to_excel('./data/다나와_무선청소기_전처리결과.xlsx', index=False)

##### 무선청소기 분석전 결측치 제거

In [ ]:
dfCleaner = pd.read_excel('./data/다나와_무선청소기_전처리결과.xlsx')

In [ ]:
dfCleaner.info()

In [ ]:
# 회사명 빠진조건
condition = dfCleaner['회사명'].isnull() == True

In [ ]:
## 회사명 빠진것 10건은 직접 수동으로 채워넣음
dfCleaner[condition]

In [ ]:
## 사용시간, 흡입력 NaN인 것은 전부 0으로 채워넣기
dfCleaner = dfCleaner.fillna(0)

In [ ]:
dfCleaner.info()

##### 필요제품 선별

In [ ]:
# 카테고리별 제품 개수 -> 워드클라우드 가능
dfCleaner['카테고리'].value_counts()

In [ ]:
dfCleaner['회사명'].value_counts()

In [ ]:
# TEST -> 워드클라우드 가능
dfTest = pd.DataFrame( dfCleaner['회사명'].value_counts())
dfTest.reset_index().head(5)

In [ ]:
## 핸드스틱청소기만 선택해서 분석
dfDataFinal = dfCleaner[dfCleaner['카테고리'].isin(['핸디스틱청소기', '핸디스틱청소기+로봇청소기', '스틱청소기', '진공청소기'])]

In [ ]:
dfDataFinal.to_excel('./data/2_danawa_data_final.xlsx', index=False)

##### 분석용 재로드

In [ ]:
dfDataFinal = pd.read_excel('./data/2_danawa_data_final.xlsx')

In [ ]:
dfDataFinal.tail()

In [ ]:
# 흡입력 기준으로 정렬, ascending=Treu(오름차순), ascending=False(내림차순)
suctionTopList = dfDataFinal.sort_values(['흡입력'], ascending=False)
suctionTopList.head()

In [ ]:
# 사용시간 기준 정렬
useTimeTopList = dfDataFinal.sort_values(['사용시간'], ascending=False)
useTimeTopList.head()

In [ ]:
# 사용시간, 흡입력 동시에 기준 정렬
topList = dfDataFinal.sort_values(['사용시간', '흡입력'], ascending=False)
topList.head()

In [ ]:
## 가성비 좋은 제품 찾기 전
# 평균값
priceMean = int(dfDataFinal['가격'].mean()) # 47만원
suctionMean = dfDataFinal['흡입력'].mean() # 115.8
useTimeMean = dfDataFinal['사용시간'].mean() # 39분
print(f'평균가격 : {priceMean:,d}원, 평균흡입력: {suctionMean:,.2f}W, 평균사용시간: {useTimeMean:,.2f}분')

In [ ]:
## 가성비 좋은 제품 조건
cond1 = dfDataFinal['가격'] <= priceMean
cond2 = dfDataFinal['흡입력'] >= suctionMean
cond3 = dfDataFinal['사용시간'] >= useTimeMean

In [ ]:
## 가성비 좋은 제품 검색
chartData = dfDataFinal[cond1 & cond2 & cond3]

##### 데이터 시각화


In [ ]:
# 필요라이브러리 사용등록
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_style('darkgrid')

In [ ]:
# 한글 깨짐 문제 해결
from matplotlib import rc

rc('font', family='D2Coding')

plt.rcParams['axes.unicode_minus'] = False

In [ ]:
## 흡입력, 사용시간 최대값, 최소값
suctionMax = chartData['흡입력'].max() # 420
useTimeMax = chartData['사용시간'].max() # 180
suctionMean2 = chartData['흡입력'].mean() # 229 
useTimeMean2 = chartData['사용시간'].mean() # 51.9

In [ ]:
suctionMean2

In [ ]:
import mplcursors
fig = plt.figure(figsize=(18, 8))

sns.scatterplot(data=chartData, x='흡입력', y='사용시간', size='가격', sizes=(10, 400), hue=chartData['회사명'], legend=False)
plt.plot([100, suctionMax], [useTimeMean2, useTimeMean2], 'r--', lw=1) # 사용시간 평균치 줄표시
plt.plot([suctionMean2, suctionMean2], [20, useTimeMax], 'b--', lw=1) # 흡입력 평균치 줄표시

mplcursors.Cursor(hover=True,artists='me')
plt.show()

In [ ]:
## 인기제품 데이터 시각화
chartDataTop = chartData[:20]

In [ ]:
import random

# 인기제품 시각화
fig = plt.figure(figsize=(20, 10))
plt.title('무선청소기 Top20')
sns.scatterplot(data=chartDataTop, x='흡입력', y='사용시간', size='가격', sizes=(20, 1000),
                hue=chartDataTop['회사명'])

for index, row in chartDataTop.iterrows():
    x = row['흡입력'] + random.randrange(-10, 10)
    y = row['사용시간'] + random.randrange(-5, 5)
    s = row['제품명']
    plt.text(x, y, s, size=13)

plt.show()

##### 결론
- 데이터분석을 위한 전처리른 쉽지 않음
- 결과들이 예상이나, 실제와 다를 수 있음